In [1]:
import pandas as pd
import matplotlib.pyplot as plt
#pd.options.display.float_format = '{:,.2f}'.format

In [2]:
file_name_1 = 'C:/temp/export-address-token-0xdc24316b9ae028f1497c275eb9192a3ea0f67022_1.csv'
file_name_2 = 'C:/temp/export-address-token-0xdc24316b9ae028f1497c275eb9192a3ea0f67022_2.csv'
df_1 = pd.read_csv(file_name_1)  
df_2 = pd.read_csv(file_name_2) 
df_steth= pd.concat([df_1, df_2],ignore_index=True)

df_steth['Value'] = pd.to_numeric(df_steth['Value'].str.replace(',', ''))
df_steth.info()


file_name_3 = 'C:/temp/export-0xdc24316b9ae028f1497c275eb9192a3ea0f67022_1.csv'
file_name_4 = 'C:/temp/export-0xdc24316b9ae028f1497c275eb9192a3ea0f67022_2.csv'
df_3 = pd.read_csv(file_name_3)  
df_4 = pd.read_csv(file_name_4) 
df_internal= pd.concat([df_3, df_4],ignore_index=True)
df_internal = df_internal[df_internal['ErrCode']!='Out of gas']
df_internal.info()

file_name_5 = 'C:/temp/export-0xdc24316b9ae028f1497c275eb9192a3ea0f67022_withnotes_1.csv'
file_name_6 = 'C:/temp/export-0xdc24316b9ae028f1497c275eb9192a3ea0f67022_withnotes_2.csv'
df_5 = pd.read_csv(file_name_5)  
df_6 = pd.read_csv(file_name_6) 
df_trans= pd.concat([df_5, df_6],ignore_index=True)
df_trans = df_trans[df_trans['ErrCode']!='Out of gas']
df_trans.info()

file_name_7 = 'C:/temp/export-token-0x06325440d014e39736583c165c2963ba99faf14e_1.csv'
file_name_8 = 'C:/temp/export-token-0x06325440d014e39736583c165c2963ba99faf14e_2.csv'
df_7 = pd.read_csv(file_name_7)  
df_8 = pd.read_csv(file_name_8) 
df_stecrv= pd.concat([df_7, df_8],ignore_index=True)
df_stecrv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5568 entries, 0 to 5567
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Txhash           5568 non-null   object 
 1   UnixTimestamp    5568 non-null   int64  
 2   DateTime         5568 non-null   object 
 3   From             5568 non-null   object 
 4   To               5568 non-null   object 
 5   Value            5568 non-null   float64
 6   ContractAddress  5568 non-null   object 
 7   TokenName        5568 non-null   object 
 8   TokenSymbol      5568 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 391.6+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5935 entries, 0 to 5934
Data columns (total 19 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Txhash                       5935 non-null   object 
 1   Blockno                      5935 non-

In [3]:
# steth sold
adr_sold_steth = df_steth[df_steth['From']!='0xdc24316b9ae028f1497c275eb9192a3ea0f67022'][['Txhash','DateTime','From','Value']] 
adr_sold_steth.columns = ['Txhash','DateTime','Address','Value_steth']
adr_sold_steth['Value_steth'] = adr_sold_steth['Value_steth']*(-1)

# steth bought
adr_bought_steth = df_steth[df_steth['To']!='0xdc24316b9ae028f1497c275eb9192a3ea0f67022'][['Txhash','DateTime','To','Value']] 
adr_bought_steth.columns = ['Txhash','DateTime','Address','Value_steth']

# steth turnover
adr_steth= pd.concat([adr_bought_steth, adr_sold_steth],ignore_index=True).sort_values(by='DateTime') 
adr_list = adr_steth['Txhash'].unique().tolist()

# eth bought (when steth sold)
adr_bought_eth  = df_internal.query('Txhash in @adr_list')[['Txhash', 'DateTime','TxTo','Value_OUT(ETH)']]
adr_bought_eth.columns = ['Txhash','DateTime','Address','Value_eth']

# eth sold (when steth bought)
adr_sold_eth = df_trans.query('Txhash in @adr_list')[['Txhash', 'DateTime','From','Value_IN(ETH)']]
adr_sold_eth.columns = ['Txhash','DateTime','Address','Value_eth']
adr_sold_eth['Value_eth'] = adr_sold_eth['Value_eth']*(-1)

# eth turnover
adr_eth= pd.concat([adr_bought_eth, adr_sold_eth],ignore_index=True).sort_values(by='DateTime') 
adr_eth = adr_eth.pivot_table(index = ['Txhash','DateTime','Address'], values = 'Value_eth', aggfunc='sum').reset_index()

# sterth + eth turnover together
adr_steth_eth = pd.merge(adr_steth,adr_eth, on = ['Txhash','DateTime','Address'])

# check add/remove liqiudity
adr_stecrv_add = df_stecrv.query('Txhash in @adr_list')[df_stecrv.query('Txhash in @adr_list')['From'] == '0x0000000000000000000000000000000000000000'][['Txhash', 'DateTime', 'To','Quantity']]
adr_stecrv_add.columns = ['Txhash','DateTime','Address','Value_stecrv']
adr_stecrv_remove = df_stecrv.query('Txhash in @adr_list')[df_stecrv.query('Txhash in @adr_list')['To'] == '0x0000000000000000000000000000000000000000'][['Txhash', 'DateTime','From','Quantity']]
adr_stecrv_remove['Quantity']=adr_stecrv_remove['Quantity'] *(-1)
adr_stecrv_remove.columns = ['Txhash','DateTime','Address','Value_stecrv']
adr_stecrv= pd.concat([adr_stecrv_add, adr_stecrv_remove],ignore_index=True).sort_values(by='DateTime') 

adr_full = pd.merge(adr_steth_eth,adr_stecrv, on = ['Txhash','DateTime','Address'], how='left')
adr_full['steth_price']=adr_full['Value_eth']/adr_full['Value_steth']*(-1)
result1=[]
result2=[]
for row in adr_full[['Value_steth','Value_eth', 'Value_stecrv']].to_numpy():
    if row[2]>0:
        result1.append('add_liquidity')
        result2.append('liquidity')
    elif row[2]<0:
        result1.append('remove_liquidity')
        result2.append('liquidity')
    elif row[0] >0 and row[1] < 0:
        result1.append('steth_bought') 
        result2.append('exchange')
    elif row[0] <0 and row[1] > 0:
        result1.append('steth_sold')
        result2.append('exchange')
    else:
        result1.append('tbd')
        result2.append('tbd')
adr_full['type_1'] =  result1 
adr_full['type_0'] =  result2 

list_sold = adr_full[adr_full['type_1']=='steth_sold']['Address'].unique().tolist()
list_bought = adr_full[adr_full['type_1']=='steth_bought']['Address'].unique().tolist()
list_ex = (set(list_sold)&set(list_bought))


adr_ex=adr_full.query('Address in @list_ex')

list_txhash_ex = adr_ex['Txhash'].tolist()
df_trans.query('Txhash in @list_txhash_ex')[['Txhash','TxnFee(ETH)']]
adr_full_ex = pd.merge(adr_ex,df_trans.query('Txhash in @list_txhash_ex')[['Txhash','TxnFee(ETH)']], on = 'Txhash')
adr_full_ex['TxnFee(ETH)'] = adr_full_ex['TxnFee(ETH)'] *(-1)
adr_full_ex.sort_values(by = ['Address','DateTime'])



,Txhash,DateTime,Address,Value_steth,Value_eth,Value_stecrv,steth_price,type_1,type_0,TxnFee(ETH)
133,0x87f64b1e48c02c9c676f6436bc6e925a2a0fcf7e3678...,2021-01-24 21:28:27,0x000ad8f56d3408abe29466189612d1b7b19e4420,5.148609,-5.000000,NaN,0.971136,steth_bought,exchange,-0.010724
144,0x5932d4624ef919eb79c184c84c7e12a0e0986b8e08ea...,2021-01-25 08:29:30,0x000ad8f56d3408abe29466189612d1b7b19e4420,-5.148609,5.114416,NaN,0.993359,steth_sold,exchange,-0.008698
251,0x145d66f132efa3ac25dc8edfda01cb8d3179751c4fee...,2021-02-10 22:28:49,0x000ad8f56d3408abe29466189612d1b7b19e4420,5.102535,-5.000000,NaN,0.979905,steth_bought,exchange,-0.012315
381,0xa3f089f772dec0edd4e223fa2c1f7e68264ae83f2d46...,2021-03-07 22:07:36,0x000ad8f56d3408abe29466189612d1b7b19e4420,5.221344,-5.000000,NaN,0.957608,steth_bought,exchange,-0.007834
448,0xf446a409bf7d734780d8e6c844c88e9942dfb779704b...,2021-04-04 21:00:22,0x000ad8f56d3408abe29466189612d1b7b19e4420,-10.407321,10.304599,NaN,0.990130,steth_sold,exchange,-0.011599
...,...,...,...,...,...,...,...,...,...,...
411,0xe341daf448d2c5219c4663c03fff539f7b49790da797...,2021-03-13 14:29:25,0xfe62145b68ae16b59f88c121651a9019f13f3653,5.421903,-5.300000,NaN,0.977516,steth_bought,exchange,-0.014379
413,0x186b2df63569ee84927043084e1e7602c791a42324ff...,2021-03-14 17:42:21,0xfe62145b68ae16b59f88c121651a9019f13f3653,-1.500000,1.461587,NaN,0.974391,steth_sold,exchange,-0.016427
475,0x83d0ce27f57035264a262ba8b8389d91c30a285cde56...,2021-04-22 22:50:14,0xfe62145b68ae16b59f88c121651a9019f13f3653,-12.261351,12.173190,NaN,0.992810,steth_sold,exchange,-0.012023
457,0x7c796c3ef9ee2422904f93f404d8846671318d839c9c...,2021-04-11 03:30:49,0xfee026defeda09cabc2675da52c0c16001651488,1.404833,-1.400000,NaN,0.996559,steth_bought,exchange,-0.011073


In [4]:
dic_resume_all = {}
list_resume_arbing = []
dic_trans = {}
list_resume_arbing_leaders = []
for address in list_ex:
    type_adr = 'exchange'
    df_adr_arb =adr_full_ex.query('Address == @address and type_0 == @type_adr')
    margin = abs(df_adr_arb[df_adr_arb['type_1']=='steth_sold']['Value_eth'].sum()/df_adr_arb[df_adr_arb['type_1']=='steth_sold']['Value_steth'].sum()) - abs(df_adr_arb[df_adr_arb['type_1']=='steth_bought']['Value_eth'].sum()/df_adr_arb[df_adr_arb['type_1']=='steth_bought']['Value_steth'].sum())
    value_steth_for_calc = min(abs(df_adr_arb[df_adr_arb['type_1']=='steth_sold']['Value_steth'].sum()),abs(df_adr_arb[df_adr_arb['type_1']=='steth_bought']['Value_steth'].sum()))
    gross_profit = value_steth_for_calc*margin
    fees_paid = df_adr_arb[df_adr_arb['type_1']=='steth_sold']['TxnFee(ETH)'].sum()*value_steth_for_calc/abs(df_adr_arb[df_adr_arb['type_1']=='steth_sold']['Value_steth'].sum()) + df_adr_arb[df_adr_arb['type_1']=='steth_bought']['TxnFee(ETH)'].sum()*value_steth_for_calc/abs(df_adr_arb[df_adr_arb['type_1']=='steth_bought']['Value_steth'].sum())
    operating_profit = gross_profit+fees_paid
    time_period = df_trans[df_trans['Txhash'] == df_adr_arb.iloc[len(df_adr_arb)-1]['Txhash']]['UnixTimestamp'].sum() - df_trans[df_trans['Txhash'] == df_adr_arb.iloc[0]['Txhash']]['UnixTimestamp'].sum()
    APR = gross_profit/value_steth_for_calc / time_period *(365*24*60*60*100)
    net_APR = operating_profit/value_steth_for_calc / time_period *(365*24*60*60*100)
    
    adr_resume = pd.DataFrame({'characteristics':['Value of exchange, stETH',
                                                 'Margin of exchange, ETH',
                                                 'Gross profit, ETH:',
                                                 'APR, %:',
                                                 'Fees paid, ETH:',
                                                 'Operating profit, ETH:',
                                                 'Net APR, %:'],
                              'values':[value_steth_for_calc,
                                       margin,
                                       gross_profit,
                                       APR,
                                       fees_paid,
                                       operating_profit,
                                       net_APR]}).set_index('characteristics')
    
    dic_resume_all.update({address:adr_resume})
    
    if margin > 0: 
        list_resume_arbing.append(address)
        
        
    if (df_adr_arb[df_adr_arb['type_1']=='steth_sold']['steth_price'].max()-df_adr_arb[df_adr_arb['type_1']=='steth_bought']['steth_price'].min()) > 0.02:
        list_resume_arbing_leaders.append(address)                                                                                                       
    
    dic_trans.update({address:df_adr_arb[['Txhash','DateTime', 'Value_steth', 'Value_eth', 'TxnFee(ETH)', 'steth_price', 'type_1']].set_index('Txhash')})
    
#    print('Value of exchange, stETH:', value_steth_for_calc)
#    print('Margin, ETH:', round(margin,6))
#    print('Gross profit, ETH:',gross_profit)
#    print('APR, %:', APR)
#    print()
#    print('Fees paid, ETH:',fees_paid)
 #   print('Operating profit, ETH:', operating_profit)
 #   print('Net APR, %:', round(net_APR,2))
 #   print()
 #   print()

In [5]:
for address in list_resume_arbing_leaders:
    print('Address:', address)
    display(dic_resume_all.get(address))
    display(dic_trans.get(address))
    print()
    print()
    print()

Address: 0xe641a99b99b7fe11d6d6b44442a2a37bfd5696e5


,values
characteristics,
"Value of exchange, stETH",20.611042
"Margin of exchange, ETH",0.025897
"Gross profit, ETH:",0.533770
"APR, %:",53.939999
"Fees paid, ETH:",-0.012188
"Operating profit, ETH:",0.521582
"Net APR, %:",52.708313


,DateTime,Value_steth,Value_eth,TxnFee(ETH),steth_price,type_1
Txhash,,,,,,
0x0de075a9f6077a18c5d8dacc1b0e9a184eb3abdc63caecffab6a1882102b09d8,2021-03-28 09:42:01,20.611042,-20.000000,-0.008676,0.970354,steth_bought
0x9e0ef0ff408004c5c9451e6f66c282ab2cbb1f5fe3f5b17ab0a12172a84341cc,2021-04-14 22:16:45,-70.871092,70.605393,-0.012077,0.996251,steth_sold





Address: 0xa91508dc7a7753ac2420adc2fbd17ce8364810e7


,values
characteristics,
"Value of exchange, stETH",6.103101
"Margin of exchange, ETH",0.012453
"Gross profit, ETH:",0.076000
"APR, %:",12.174912
"Fees paid, ETH:",-0.032419
"Operating profit, ETH:",0.043580
"Net APR, %:",6.981435


...

,values
characteristics,
"Value of exchange, stETH",160.256579
"Margin of exchange, ETH",0.008277
"Gross profit, ETH:",1.326468
"APR, %:",4.079201
"Fees paid, ETH:",-0.106349
"Operating profit, ETH:",1.220119
"Net APR, %:",3.752152


,DateTime,Value_steth,Value_eth,TxnFee(ETH),steth_price,type_1
Txhash,,,,,,
0xaf078f498be7467f6bcfb31a0f3edcf7914dcf008e6b88e536a327cb91c870cc,2021-01-24 21:37:22,47.250526,-46.000000,-0.009712,0.973534,steth_bought
0x0075a46728941b0f2c77200c5b5033fd5906151b9316ea51abfd8bd5baf9fb67,2021-01-26 05:52:20,-47.252949,47.124238,-0.007727,0.997276,steth_sold
0x2ce163a5f12c7acfaa589c4c100323f510ba6651b276e0f5ddf9b03df4060d9f,2021-02-10 05:37:03,23.104526,-23.000000,-0.013097,0.995476,steth_bought
0x32ed9fef2b007cf503b5293fb2c1ebab37d521be9161f1db3df372d400e47cb2,2021-02-14 19:22:47,-23.119321,23.065859,-0.016418,0.997688,steth_sold
0xe6bd25f69b387ab87a0cdac9cb4bc6a3c4539bfd3188ebf1b764a8f8f8cfe99a,2021-02-18 04:34:37,35.336362,-35.000000,-0.014133,0.990481,steth_bought
0xcdb24b0c8976436f936a395d74f135bffc32dd14d4424ae34424ed70aa4e6d32,2021-02-19 14:50:54,-35.342102,35.313003,-0.022919,0.999177,steth_sold
0xe1344536371e6f5e639ace44164ed150231e3b03507030d94568afa94ea14915,2021-02-21 06:40:32,60.002874,-59.520000,-0.011095,0.991952,steth_bought
0xd735c89cfdb55ad92c8f55c00792646ba2ada63687aa1b17a4443ea0ac10de26,2021-04-08 23:07:28,-54.542206,53.977014,-0.012824,0.989638,steth_sold





Address: 0x40d227723d66eb5d5cb1dff427c51a49d0b9957e


,values
characteristics,
"Value of exchange, stETH",1590.818500
"Margin of exchange, ETH",0.005491
"Gross profit, ETH:",8.734513
"APR, %:",7.645423
"Fees paid, ETH:",-0.324218
"Operating profit, ETH:",8.410295
"Net APR, %:",7.361631


,DateTime,Value_steth,Value_eth,TxnFee(ETH),steth_price,type_1
Txhash,,,,,,
0x6897a7e3a7d4c0a7eba14106a75b2f19c55e6a8f5943691c641b11faf0482950,2021-01-24 20:29:32,104.371567,-100.000000,-0.012968,0.958115,steth_bought
0x0549b6e99a8e9f2f5be29c8676275545493bd9f9f07d9ae54523b70e033492ed,2021-01-31 05:37:06,101.014453,-100.000000,-0.013651,0.989957,steth_bought
0x71ce5216adb98fb8b9ebdd20b7375b9d964b6e76cdbb9d2063126d2c907c7b84,2021-02-01 05:36:54,-100.152000,100.001346,-0.013553,0.998496,steth_sold
0x7b424e71cddc5de2dd3668a1516dccc79f7e041c920ca9308c59b763b8590077,2021-02-07 00:40:11,250.478704,-250.000000,-0.016182,0.998089,steth_bought
0xe1a6012fd4f03f54a024f85022b6d371e6f52dbec44eda8f4b79dd7ab338c4b0,2021-02-07 02:49:00,-250.000000,249.907013,-0.024642,0.999628,steth_sold
0x7f8be0dc5fd7e37afcb61d997b82b7c55ed7dae2b1f61df957f0af5b88b8c581,2021-02-09 22:58:05,200.448440,-200.000000,-0.026971,0.997763,steth_bought
0x9587e435bd7ef91f8af5304b0af356468fb1bc454b3acba172e9c39dc44511a1,2021-02-10 00:06:16,-200.126500,200.000055,-0.030805,0.999368,steth_sold
0x9c53c08512ec2c7d48fd7f9819f6a3d07e488d1c4bbd736b2f332ba909742613,2021-02-10 03:35:59,250.991916,-250.000000,-0.026971,0.996048,steth_bought
0x91bacd9d135a8c84bbdf9574538c6804b94a2e0b7c6a0c0d1beff4441e90310f,2021-02-13 00:22:43,-250.060000,250.004901,-0.034434,0.999780,steth_sold


In [6]:
adr_arb_empirical = [
'0x49cae5c4e03196bef2f9984469bd096878043719',
'0xe1e2a52a21226ba37f28ba6cdf59f20156798f94',
'0x125955cd603650eb3b1c0cf000ab78ad0213ddaf',
'0xa2e83e57502a89a3af7ceacc195b320f9ad7d64d',
'0x75c0c0591cdf9af7cb826a65ac213cff31916668',
'0x40d227723d66eb5d5cb1dff427c51a49d0b9957e',
'0x311bf6e6b6d416a808adf524b2fba791bf257e19',
'0x37ad9f5cb15867156bd75e76b8d9edc40fc077a8',
'0x97960149fc611508748de01202974d372a677632',
'0x6a77500a141fede525e57e422d65d67a32f2d8fe',
'0x116c095dc2b9fd304f5b20ea3a051ac82dd61c3a',
'0xdd01596074fa8e2a2f84d10d89090ae4543ea20b',
'0xe641a99b99b7fe11d6d6b44442a2a37bfd5696e5',
'0xec5abba31c186a9f2695a66a734f3d3ad6e85313',
'0xfe62145b68ae16b59f88c121651a9019f13f3653',
'0x000ad8f56d3408abe29466189612d1b7b19e4420',
'0xfaf16f5fd569ca0a22fd6a90cbd84bb18f5e2882',
'0xd9d41ad0fde1a199a5ecc896fc33806b189c1d0f',
'0xbfb0d8b06ec79d5f2059209221b15644956e8261',
'0xa40eb69d2adbd34ea73ea1dfd049f825e8216eb0',
'0xc50eaaee441566b4ef4f1e3270d19f24223cddf4',
'0x50013c043ea8ed2b5377e38ff5a874dbfd16460b',
'0xbfb496acb99299e9ece84b3fd1b3fdd0f6cddf49',
'0xa2d6db7b72e32091e9aa6514a57f8dc94971cbe0',
'0xcbba4fb07d4a86267741cf1a2c363284eed00f0f',
'0x11f262d4154f7860da5e4f595778087eb7434bce',
'0x0e66cf7a7e7a7f4574670f6c2c9d463e1478a06c',
'0x0eb4add4ba497357546da7f5d12d39587ca24606',
'0x1488ec6c55243e7130b756949b0238693c7e382b'

]
len(adr_arb_empirical)

29